In [ ]:
using Pkg
Pkg.activate(".")
Pkg.develop(path="../../Tambo/")
using Tambo

In [ ]:
using JLD2
using Statistics
using Glob
using Makie
using LaTeXStrings
using CairoMakie

In [ ]:
mountain_cmap=cgrad([
    :navajowhite3,
    :goldenrod4,
    :olivedrab,
    :green]
)

time_cmap = cgrad(:rainbow, rev=true)

In [ ]:
sim_example = jldopen("../../ack_scratch/sims/larger_valley_00000_00000.jld2")
sim_config = Dict(kv[1]=>kv[2] for kv in sim_example["config"])

geo_config = Dict(
    "geo_spline_path" => "../../resources/tambo_spline.jld2",
    "tambo_coordinates" => rad2deg.([sim_config[:tambo_coordinates].latitude, sim_config[:tambo_coordinates].longitude]),
    "plane_orientation" => sim_example["config"][:plane_orientation].proj
)

geo = Tambo.Geometry(geo_config)
plane = Tambo.Plane(normal_vecs.larger_valley, Tambo.coords.larger_valley, geo)

## First, we have to load up the data, and make our array

## Utility functions

### Finds the time window for an event

In [ ]:
function find_time_range(hit_map)
    tmin, tmax = Inf, -Inf
    for (idx, evts) in hit_map
        for evt in evts
            tmin = minimum([evt.time, tmin])
            tmax = maximum([evt.time, tmax])
        end
    end
    return tmin, tmax
end

In [ ]:
ℓ = 2000
fs = glob("larger_valley_hard_00000_000??_150_2000.jld2", "/Users/jlazar/Downloads/")
f = fs[2]
event_number = 5097
jldf = jldopen(f)
hit_map = jldf["hit_map"][string(event_number)]
modules = Tambo.make_detector_array(
    jldf["array_config"]["length"],
    jldf["array_config"]["deltas"],
    jldf["array_config"]["altmin"],
    jldf["array_config"]["altmax"],
    plane,
    geo,
    Tambo.SVector{3}(1,2,1)
)
sim = jldopen("../../ack_scratch/sims/larger_valley_00000_00002.jld2")
eini = sim["injected_events"][event_number].entry_state.energy / units.GeV
eout = sum(map(x->x.energy, filter(x->x.pdg_mc!=16, sim["proposal_events"][event_number].decay_products))) / units.GeV

In [ ]:
xmin=-2.5units.km
xmax=2.5units.km
ymin=-2.5units.km
ymax=2.5units.km

module_color = colorant"black"
module_size = 10.0
sizescaler = 7.0

contour_xs = LinRange(xmin, xmax, 201)
contour_ys = LinRange(ymin, ymax, 200)
ncountour = 10

plot_xs = rad2deg.([Tambo.xy_to_latlong([x + geo.tambo_offset[1], 0], geo.valley.mincoord).longitude for x in contour_xs])
plot_ys = rad2deg.([Tambo.xy_to_latlong([0, y + geo.tambo_offset[2]], geo.valley.mincoord).latitude for y in contour_ys])

set_theme!(fonts = (; regular = "Times"))
fig = Makie.Figure(size=(600, 600), fontsize=18)
ax = Axis(
    fig[2, 1],
    xlabel=L"\mathrm{Longitude}",
    ylabel=L"\mathrm{Latitude}",
    xticks=(-72.3:0.01:-72.26, [L"%$(abs(x))^{\circ}\,\mathrm{W}" for x in -72.3:0.01:-72.26]),
    yticks=(-15.6:-0.01:-15.64, [L"%$(abs(x))^{\circ}\,\mathrm{S}" for x in -15.6:-0.01:-15.64]),
#     xticklabelrotation=deg2rad(10)
)

# Plot the filled contour region region
ctrf = Makie.contourf!(
    ax,
    plot_xs,
    plot_ys,
    [(geo(x, y) + geo.tambo_offset[3]) / units.km for x in contour_xs, y in contour_ys],
    colormap=mountain_cmap,
    levels=ncountour
)

# Plot the lines
Makie.contour!(
    ax,
    plot_xs,
    plot_ys,
    [(geo(x, y) + geo.tambo_offset[3]) / units.km for x in contour_xs, y in contour_ys],
    color=:black,
    levels=ncountour-1
)

colors = fill(module_color, length(modules))
markersize = fill(module_size, length(modules))
times = zeros(length(modules))
        
nmax = 0
if ~(hit_map==nothing)
    tmin, tmax = find_time_range(hit_map)
    hit_color(t) = get(time_cmap, (t - tmin) / (tmax - tmin))
    for (idx, evts) in hit_map
#         evts = filter(e->e.time<tcut, evts)
        if length(evts)==0
            continue
        end
        times[idx] = mean(getfield.(evts, :time))
        colors[idx] = hit_color(mean(getfield.(evts, :time)))
        nmax = maximum([nmax, length(evts)])
        markersize[idx] = module_size + sizescaler * log(10, length(evts))
    end
end

m = colors.==colorant"black"

module_coords = map(
    x->Tambo.xy_to_latlong([x.pos[1], x.pos[2]].+geo.tambo_offset[1:2], geo.valley.mincoord),
    modules
)
    
module_xs = map(x->rad2deg(x.longitude), module_coords)
module_ys = map(x->rad2deg(x.latitude), module_coords)

# Plot untouched modules
Makie.scatter!(
    ax,
    module_xs[m],
    module_ys[m],
    alpha=0.3,
    markersize=markersize[m],
    color=colors[m],
)

# Plot the modules that had hits, with high zorder ~ later times
xs_ = module_xs[.~m]
ys_ = module_ys[.~m]
ts_ = times[.~m]
ss_ = markersize[.~m]
cs_ = colors[.~m]
sorter = sortperm(ts_)
xs_, ys_, ss_, cs_ = xs_[sorter], ys_[sorter], ss_[sorter], cs_[sorter]
    
sct = Makie.scatter!(
    ax,
    xs_,
    ys_,
    alpha=1.0,
    markersize=ss_,
    color=cs_,
    strokewidth = 0.5,
)

# Set up the colorbars for elevationa and timing
cbar = Colorbar(fig[2, 2], ctrf, vertical=true, label=L"\mathrm{Elevation}~\left[\mathrm{km}\right]")
tbar = Colorbar(
    fig[1, 1],
    limits=(0, (tmax - tmin) / units.second * 1e6),
    vertical=false,
    colormap=time_cmap,
    label=L"\mathrm{Mean}\,\mathrm{time}~[\mu \mathrm{s}]"
)

# Set up km scale marker
km_markers = map(
    x->Tambo.xy_to_latlong(x.+geo.tambo_offset[1:2], geo.valley.mincoord),
    [[1.22, -2.2].*units.km, [2.22, -2.2].*units.km]
)

km_marker_xs = map(x->rad2deg(x.longitude), km_markers)
km_marker_ys = map(x->rad2deg(x.latitude), km_markers)

lines!(ax, km_marker_xs, km_marker_ys, linewidth=5, color=:black)
for x in km_marker_xs
    lines!(ax, [x, x], [km_marker_ys[1]-1e-3, km_marker_ys[1]+1e-3], linewidth=5, color=:black)
end

text!(
    ax.scene,
    (mean(km_marker_xs), km_marker_ys[1]+5e-4),
#     justification=:center,
    text=L"1\,\mathrm{km}",
    align = (:center, :baseline)
)

# Make label with energies
poly!(
    Point2f[(-72.302, -15.6), (-72.282, -15.6), (-72.282, -15.6053), (-72.302, -15.6053)],
    color=(:white, 0.7),
    strokecolor=:black,
    strokewidth = 1
)

text!(
    ax.scene, (0.025, 0.933); # plotting in relative coordinates - (0, 0) is bottom left, (1, 1) is top right
    space = :relative, text = L"E_{\nu} = 8.16\times 10^6\,\mathrm{GeV}", color=:black, font = "Times",
    fontsize=18
)
text!(
    ax.scene, (0.025, 0.88); # plotting in relative coordinates - (0, 0) is bottom left, (1, 1) is top right
    space = :relative, text = L"E_{s} = 2.13\times 10^6\,\mathrm{GeV}", color=:black, font = "Times",
    fontsize=18
)

# Make legend for sizes of dots
exps = 0:Int(round(log(10, nmax)))
markersizes = [module_size + sizescaler * exp for exp in exps]
labels = [L"10^{%$(exp)}" for exp in exps]

group_size = [MarkerElement(marker = :circle, color = :black,
    strokecolor = :transparent,
    markersize = ms) for ms in markersizes]

legends = Legend(
    fig[2, 1],
    [group_size],
    [labels],
    [L"N_{\mathrm{particle}}"],
    tellheight = false,
    tellwidth = false,
    margin=(5, 5, 5, 5),
    halign=:left,
    valign=:bottom,
    orientation=:horizontal,
    backgroundcolor=(:white, 0.7)
)


save("../../figures/event_display.svg", fig)
display(fig)

## Make gif

In [ ]:
xmin=-2.5units.km
xmax=2.5units.km
ymin=-2.5units.km
ymax=2.5units.km

module_color = colorant"black"
module_size = 10.0
sizescaler = 7.0

contour_xs = LinRange(xmin, xmax, 201)
contour_ys = LinRange(ymin, ymax, 200)
ncountour = 10

plot_xs = rad2deg.([Tambo.xy_to_latlong([x + geo.tambo_offset[1], 0], geo.valley.mincoord).longitude for x in contour_xs])
plot_ys = rad2deg.([Tambo.xy_to_latlong([0, y + geo.tambo_offset[2]], geo.valley.mincoord).latitude for y in contour_ys])

set_theme!(fonts = (; regular = "Times"))
    
tmin, tmax = find_time_range(hit_map)

nmax = 0
if ~(hit_map==nothing)
    hit_color(t) = get(time_cmap, (t - tmin) / (tmax - tmin))
    for (idx, evts) in hit_map
        if length(evts)==0
            error("Null hitmap")
        end
        nmax = maximum([nmax, length(evts)])
    end
end
    
for (idx, tcut) in enumerate(LinRange(tmin, tmax, 500))
    fig = Makie.Figure(size=(600, 600), fontsize=18)
    ax = Axis(
        fig[2, 1],
        xlabel=L"\mathrm{Longitude}",
        ylabel=L"\mathrm{Latitude}",
        xticks=(-72.3:0.01:-72.26, [L"%$(abs(x))^{\circ}\,\mathrm{W}" for x in -72.3:0.01:-72.26]),
        yticks=(-15.6:-0.01:-15.64, [L"%$(abs(x))^{\circ}\,\mathrm{S}" for x in -15.6:-0.01:-15.64]),
    )

    # Plot the filled contour region region
    ctrf = Makie.contourf!(
        ax,
        plot_xs,
        plot_ys,
        [(geo(x, y) + geo.tambo_offset[3]) / units.km for x in contour_xs, y in contour_ys],
        colormap=mountain_cmap,
        levels=ncountour
    )

    # Plot the lines
    Makie.contour!(
        ax,
        plot_xs,
        plot_ys,
        [(geo(x, y) + geo.tambo_offset[3]) / units.km for x in contour_xs, y in contour_ys],
        color=:black,
        levels=ncountour-1
    )

    colors = fill(module_color, length(modules))
    markersize = fill(module_size, length(modules))
    times = zeros(length(modules))

    if ~(hit_map==nothing)
        hit_color(t) = get(time_cmap, (t - tmin) / (tmax - tmin))
        for (idx, evts) in hit_map
            evts = filter(e->e.time < tcut, evts)
#             @show evts
            if length(evts)==0
                continue
            end
            times[idx] = mean(getfield.(evts, :time))
            colors[idx] = hit_color(mean(getfield.(evts, :time)))
            markersize[idx] = module_size + sizescaler * log(10, length(evts))
        end
    end

    if nmax==0
        continue
    end

    m = colors.==colorant"black"

    module_coords = map(
        x->Tambo.xy_to_latlong([x.pos[1], x.pos[2]].+geo.tambo_offset[1:2], geo.valley.mincoord),
        modules
    )

    module_xs = map(x->rad2deg(x.longitude), module_coords)
    module_ys = map(x->rad2deg(x.latitude), module_coords)

    # Plot untouched modules
    Makie.scatter!(
        ax,
        module_xs[m],
        module_ys[m],
        alpha=0.3,
        markersize=markersize[m],
        color=colors[m],
    )

    # Plot the modules that had hits, with high zorder ~ later times
    xs_ = module_xs[.~m]
    ys_ = module_ys[.~m]
    ts_ = times[.~m]
    ss_ = markersize[.~m]
    cs_ = colors[.~m]
    sorter = sortperm(ts_)
    xs_, ys_, ss_, cs_ = xs_[sorter], ys_[sorter], ss_[sorter], cs_[sorter]

    sct = Makie.scatter!(
        ax,
        xs_,
        ys_,
        alpha=1.0,
        markersize=ss_,
        color=cs_,
        strokewidth = 0.5,
    )

    # Set up the colorbars for elevationa and timing
    cbar = Colorbar(fig[2, 2], ctrf, vertical=true, label=L"\mathrm{Elevation}~\left[\mathrm{km}\right]")
    tbar = Colorbar(
        fig[1, 1],
        limits=(0, (tmax - tmin) / units.second * 1e6),
        vertical=false,
        colormap=time_cmap,
        label=L"\mathrm{Mean}\,\mathrm{time}~[\mu \mathrm{s}]"
    )

    # Set up km scale marker
    km_markers = map(
        x->Tambo.xy_to_latlong(x.+geo.tambo_offset[1:2], geo.valley.mincoord),
        [[1.22, -2.2].*units.km, [2.22, -2.2].*units.km]
    )

    km_marker_xs = map(x->rad2deg(x.longitude), km_markers)
    km_marker_ys = map(x->rad2deg(x.latitude), km_markers)

    lines!(ax, km_marker_xs, km_marker_ys, linewidth=5, color=:black)
    for x in km_marker_xs
        lines!(ax, [x, x], [km_marker_ys[1]-1e-3, km_marker_ys[1]+1e-3], linewidth=5, color=:black)
    end

    text!(
        ax.scene,
        (mean(km_marker_xs), km_marker_ys[1]+5e-4),
    #     justification=:center,
        text=L"1\,\mathrm{km}",
        align = (:center, :baseline)
    )

    # Make label with energies
    poly!(
        Point2f[(-72.302, -15.6), (-72.282, -15.6), (-72.282, -15.6053), (-72.302, -15.6053)],
        color=(:white, 0.7),
        strokecolor=:black,
        strokewidth = 1
    )

    text!(
        ax.scene,
        (0.025, 0.933);
        space=:relative,
        text = L"E_{\nu} = 8.16\times 10^6\,\mathrm{GeV}",
        color=:black,
        font = "Times",
        fontsize=18
    )
    text!(
        ax.scene,
        (0.025, 0.88);
        space=:relative,
        text = L"E_{s} = 2.13\times 10^6\,\mathrm{GeV}",
        color=:black,
        font="Times",
        fontsize=18
    )

    # Make legend for sizes of dots
    exps = 0:Int(round(log(10, nmax)))
    markersizes = [module_size + sizescaler * exp for exp in exps]
    labels = [L"10^{%$(exp)}" for exp in exps]

    group_size = [MarkerElement(marker = :circle, color = :black,
        strokecolor = :transparent,
        markersize = ms) for ms in markersizes]

    legends = Legend(
        fig[2, 1],
        [group_size],
        [labels],
        [L"N_{\mathrm{particle}}"],
        tellheight = false,
        tellwidth = false,
        margin=(5, 5, 5, 5),
        halign=:left,
        valign=:bottom,
        orientation=:horizontal,
        backgroundcolor=(:white, 0.7)
    )


    save("../../figures/event_display_gif_$(lpad(idx, 4, "0")).png", fig)
end